# Train an SVM classifier on the MNIST dataset. 

Since SVM classifiers are binary classifiers, you will need to use one-versus-all to classify all 10 digits. You may want to tune the hyperparameters using small validation sets to speed up the process. What accuracy can you reach?_

First, let's load the dataset and split it into a training set and a test set. We could use `train_test_split()` but people usually just take the first 60,000 instances for the training set, and the last 10,000 instances for the test set (this makes it possible to compare your model's performance with others): 
Don't forget to convert target to int.

In [1]:
# Librerías de análisis
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import io
import requests

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

# Liberías de Visualización
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import pickle

print('ready')

ready


In [3]:
# Clases para el ML

# Para valores NUMERICOS
class ColumnsSelector(BaseEstimator, TransformerMixin):
  
  def __init__(self, type):
    self.type = type
  
  def fit(self, X, y=None):
    return self
  
  def transform(self,X):
    return X.select_dtypes(include=[self.type])


# Para valores CATEGORICOS
class CategoricalEncoder(BaseEstimator, TransformerMixin):
  
  def __init__(self, dropFirst=True):
    self.categories=dict()
    self.dropFirst=dropFirst
    
  def fit(self, X, y=None):
    join_df = pd.concat([train_data, test_data])
    join_df = join_df.select_dtypes(include=['object'])
    for column in join_df.columns:
      self.categories[column] = join_df[column].value_counts().index.tolist()
    return self
    
  def transform(self, X):
    X_copy = X.copy()
    X_copy = X_copy.select_dtypes(include=['object'])
    for column in X_copy.columns:
      X_copy[column] = X_copy[column].astype({column: CategoricalDtype(self.categories[column])})
    return pd.get_dummies(X_copy, drop_first=self.dropFirst)


class CategoricalImputer(BaseEstimator, TransformerMixin):
  
  def __init__(self, columns = None, strategy='most_frequent'):
    self.columns = columns
    self.strategy = strategy
    
    
  def fit(self,X, y=None):
    if self.columns is None:
      self.columns = X.columns
    
    if self.strategy is 'most_frequent':
      self.fill = {column: X[column].value_counts().index[0] for column in self.columns}
    else:
      self.fill ={column: '0' for column in self.columns}
      
    return self
      
  def transform(self,X):
    X_copy = X.copy()
    for column in self.columns:
      X_copy[column] = X_copy[column].fillna(self.fill[column])
    return X_copy

In [4]:
def null_count (df):
    """
                        ---What it does---
    Identifies and counts the number of null values in any given df. Does not return anything.

                        ---What it needs---
    A DataFrame
    """
    null_in_df = df.isnull().any()
    is_null = df.isnull().sum()
    print (f'Presence of null in clolumns:\n{null_in_df}\n\nNumber of null in columns:\n{is_null}')

In [18]:
mmist_train = pd.read_csv(r'F:\Programacion\1.BOOTCAMP\data\general_dfs\mnist_train.csv')
mmist_test = pd.read_csv(r'F:\Programacion\1.BOOTCAMP\data\general_dfs\mnist_test.csv')

In [19]:
mmist_train.head(1)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
mmist_test.head(1)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
mmist_test['1x1'].value_counts()

0    10000
Name: 1x1, dtype: int64

In [5]:
null_count(mmist_test), null_count(mmist_train)

Presence of null in clolumns:
label    False
1x1      False
1x2      False
1x3      False
1x4      False
         ...  
28x24    False
28x25    False
28x26    False
28x27    False
28x28    False
Length: 785, dtype: bool

Number of null in columns:
label    0
1x1      0
1x2      0
1x3      0
1x4      0
        ..
28x24    0
28x25    0
28x26    0
28x27    0
28x28    0
Length: 785, dtype: int64
Presence of null in clolumns:
label    False
1x1      False
1x2      False
1x3      False
1x4      False
         ...  
28x24    False
28x25    False
28x26    False
28x27    False
28x28    False
Length: 785, dtype: bool

Number of null in columns:
label    0
1x1      0
1x2      0
1x3      0
1x4      0
        ..
28x24    0
28x25    0
28x26    0
28x27    0
28x28    0
Length: 785, dtype: int64


(None, None)

In [16]:
mmist_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 359.3 MB


In [21]:
mmist_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 785 entries, label to 28x28
dtypes: int64(785)
memory usage: 59.9 MB


Many training algorithms are sensitive to the order of the training instances, so it's generally good practice to shuffle them first. However, the dataset is already shuffled, so we do not need to do it.

Let's start simple, with a linear SVM classifier. It will automatically use the One-vs-All (also called One-vs-the-Rest, OvR) strategy, so there's nothing special we need to do. Use a LinearSVC and initialize it with a randon_state to make results reproducible.

**Warning**: this may take a few minutes depending on your hardware.

In [2]:
pca = PCA(n_components=150, svd_solver='randomized', whiten=True, random_state=42)
svc = SVC(kernel='rbf', class_weight='balanced')
model = make_pipeline(pca, svc)

In [3]:
Xtrain, Xtest, ytrain, ytest = train_test_split(faces.data, faces.target,
                                                random_state=42)

NameError: name 'train_test_split' is not defined

Let's make predictions on the training set and measure the accuracy (we don't want to measure it on the test set yet, since we have not selected and trained the final model yet):

Okay, Probably you got an accuracy bellow 90%. On MNIST that is pretty bad. This linear model is certainly too simple for MNIST, but perhaps we just needed to scale the data first. Use and StandardScaler to do it:
Hint: take into account that you should convert the features to float. 

Try to train the model again on the scaled features.
**Warning**: this may take a few minutes depending on your hardware.

Let's make predictions on the training set and measure the accuracy 

0.9225333333333333

That's much better (we cut the error rate by about 25%), but still not great at all for MNIST. If we want to use an SVM, we will have to use a kernel. Let's try an `SVC` with an RBF kernel (the default).
#### **Let's train the model only on 10000 samples**  

**Note**: to be future-proof set `gamma="scale"` since it will be the default value in Scikit-Learn 0.22.


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Let's make predictions on the training set and measure the accuracy 

0.9455333333333333

That's promising, we get better performance even though we trained the model on 6 times less data. Let's tune the hyperparameters by doing a randomized search with cross validation. We will do this on a small dataset (1000 samples) just to speed up the process:
Use a RandomizedSearchCV

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}


Let's print the best estimator and the best score

This looks pretty low but remember we only trained the model on 1,000 instances. Let's retrain the best estimator on the whole training set (**run this at night, it will take hours**):

Let's make predictions on the training set and measure the accuracy 

Ah, this looks good! Let's select this model. Now we can test it on the test set:

Not too bad, but apparently the model is overfitting slightly. It's tempting to tweak the hyperparameters a bit more (e.g. decreasing `C` and/or `gamma`), but we would run the risk of overfitting the test set. Other people have found that the hyperparameters `C=5` and `gamma=0.005` yield even better performance (over 98% accuracy). By running the randomized search for longer and on a larger part of the training set, you may be able to find this as well.

## Train an SVM regressor on the California housing dataset

Let's load the dataset using Scikit-Learn's `fetch_california_housing()` function:

Split it into a training set and a test (20%) set:

Don't forget to scale the data. Use a StandardScaler

In [0]:

X_test_scaled = scaler.transform(X_test)

Let's train a simple `LinearSVR` first:

Let's see how it performs on the training set using the Mean Squared Error:

Let's look at the RMSE:

In this training set, the targets are tens of thousands of dollars. The RMSE gives a rough idea of the kind of error you should expect (with a higher weight for large errors): so with this model we can expect errors somewhere around $10,000. Not great. Let's see if we can do better with an RBF Kernel. We will use randomized search with cross validation to find the appropriate hyperparameter values for `C` and `gamma`:

Let's see the best estimator.

Now let's measure the RMSE  and the squared error on the training set for this estimator:

Looks much better than the linear model. Let's select this model and evaluate it on the test set: